In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [7]:
spark = (
    SparkSession
    .builder
    .appName("enade_app")
    .config("extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true,-Divy.cache.dir=/tmp,-Divy.home=/tmp")
    .config("spark.executor.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [8]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("fs.s3a.fast.upload:", "true")

In [9]:
BUCKET_LANDING = "dl-landing-zone-401868797180"
BUCKET_PROCESSING = "dl-processing-zone-401868797180"

In [10]:
df_landing = (
    spark
    .read
    .format("csv")
    .options(header='true', inferSchema='true', delimiter=';')
    .load(f"s3a://{BUCKET_LANDING}/enade2017/microdados2017_arq1.txt")
)

Py4JJavaError: An error occurred while calling o51.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:752)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da

In [ ]:
df_landing.printSchema()

In [ ]:
df_landing.count()

In [ ]:
df_processing = (
    spark
    .read
    .format("parquet")
    .load(f"s3a://{BUCKET_PROCESSING}/combustiveis/")
)

In [ ]:
df_processing.printSchema()

In [ ]:
df_processing.count()

In [ ]:
df_processing_2016_01 =  df_processing.where(f.col('file_name') == f's3a://{BUCKET_LANDING}/combustiveis/ca-2016-01.csv')

In [ ]:
(
    df_processing_2016_01
    .select("estado_sigla", "valor_de_venda", "valor_de_compra")
    .groupBy('estado_sigla')
    .mean().show(30)
)